# Project 3 Part 2 - Specifications - Financial Data (API Calls)

In [57]:
import os, time,json
import tmdbsimple as tmdb 
import pandas as pd
import requests
from io import BytesIO
import gzip
import json

In [50]:
# Load API Credentials
with open('/Users/MiPillay1/OneDrive - MMI HOLDINGS LTD/Documents/GitHub/Project_3-Data_Enrichment/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)


In [51]:
login.keys()

dict_keys(['client-id', 'api-key'])

In [52]:
tmdb.API_KEY =  login['api-key']

In [53]:
def get_movie_data(movie_id):
    try:

        # Create a Movie object for the given movie ID
        movie = tmdb.Movies(movie_id)

        # Retrieve movie information
        movie_info = movie.info()

        # Retrieve releases to get the certification (MPAA rating)
        releases = movie.releases()

        for c in releases['countries']:
            if c['iso_3166_1'] == 'US':
                movie_info['certification'] = c['certification']

        return movie_info
    except Exception as e:
        print(f"Error getting movie data for movie {movie_id}: {e}")
        return None

In [54]:
def write_json(new_data, filename):
    try:
        # Ensure the directory exists before writing the JSON file
        os.makedirs(os.path.dirname(filename), exist_ok=True)

        with open(filename, 'r+') as file:
            file_data = json.load(file)
            if isinstance(new_data, list) and isinstance(file_data, list):
                file_data.extend(new_data)
            else:
                file_data.append(new_data)
            file.seek(0)
            json.dump(file_data, file, indent=4)
    except Exception as e:
        print(f"Error writing to JSON file {filename}: {e}")

In [55]:
# Test the API function on sample movie IDs
sample_movie_ids = ["tt0848228", "tt0332280"]
movie_data = []

for movie_id in sample_movie_ids:
    movie_info = {}
    try:
        movie_info = get_movie_data(movie_id)
        movie_data.append(movie_info)
    except Exception as e:
        print(f"Error processing movie {movie_id}: {e}")

# Print the results
print("Movie data with financial information:")
print(movie_data)

Movie data with financial information:
[{'adult': False, 'backdrop_path': '/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg', 'belongs_to_collection': {'id': 86311, 'name': 'The Avengers Collection', 'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg', 'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'}, 'budget': 220000000, 'genres': [{'id': 878, 'name': 'Science Fiction'}, {'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}], 'homepage': 'https://www.marvel.com/movies/the-avengers', 'id': 24428, 'imdb_id': 'tt0848228', 'original_language': 'en', 'original_title': 'The Avengers', 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!', 'popularity': 118.426, 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg', 'production_companies': [{'id': 4

In [56]:
pd.DataFrame(movie_data)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,tt0848228,en,The Avengers,...,1518815515,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,False,7.711,29286,PG-13
1,False,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,None,29000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036,tt0332280,en,The Notebook,...,115603229,123,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,False,7.881,10694,PG-13


In [40]:
# Define the years you want to extract
years_to_extract = [2000, 2001]

In [41]:
# Create a directory to store the results
result_folder = "movie_results"
os.makedirs(result_folder, exist_ok=True)

In [46]:
# Load the filtered data from the specified path
data_file_path = "C:/Users/MiPillay1/OneDrive - MMI HOLDINGS LTD/Documents/GitHub/Project_3-Data_Enrichment/Data/filtered_title_basics.csv.gz"
basics = pd.read_csv(data_file_path, sep='\t', low_memory=False)
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97815 entries, 0 to 97814
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          97815 non-null  object 
 1   titleType       97815 non-null  object 
 2   primaryTitle    97815 non-null  object 
 3   originalTitle   97815 non-null  object 
 4   isAdult         97815 non-null  int64  
 5   startYear       97815 non-null  float64
 6   endYear         97815 non-null  object 
 7   runtimeMinutes  97815 non-null  object 
 8   genres          97815 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 6.7+ MB


In [48]:
# Load the filtered data from the specified path
data_file_path = "C:/Users/MiPillay1/OneDrive - MMI HOLDINGS LTD/Documents/GitHub/Project_3-Data_Enrichment/Data/filtered_title_basics.csv.gz"
basics = pd.read_csv(data_file_path, sep='\t', low_memory=False)

# Define the years of interest
years_to_extract = [2000, 2001]

# Create a folder to store the results
result_folder = "movie_results"
os.makedirs(result_folder, exist_ok=True)

tmdb.API_KEY =  login['api-key']

# Initialize a list to store movie data
all_movie_data = []

# Your function to add certification (MPAA rating) to movie information
def add_certification(movie_info, movie_id):
    try:
        movie = tmdb.Movies(movie_id)
        info = movie.info()
        releases = movie.releases()

        for c in releases['countries']:
            if c['iso_3166_1'] == 'US':
                movie_info['certification'] = c['certification']

        return movie_info
    except Exception as e:
        print(f"Error getting certification for movie {movie_id}: {e}")
        return movie_info

# Your function to append/extend a JSON file
def write_json(new_data, filename):
    try:
        with open(filename, 'r+') as file:
            file_data = json.load(file)
            if isinstance(new_data, list) and isinstance(file_data, list):
                file_data.extend(new_data)
            else:
                file_data.append(new_data)
            file.seek(0)
            json.dump(file_data, file)
    except Exception as e:
        print(f"Error writing to JSON file {filename}: {e}")


for year in years_to_extract:
    # Filter movies for the specified year
    filtered_data = basics[basics['startYear'] == year]
    
    # Fetch movie data for the specific movie IDs
    movie_data = []
    for movie_id in filtered_data['tconst']:
        try:
            movie_info = get_movie_data(movie_id)
            if movie_info is not None:
                movie_info = add_certification(movie_info, movie_id)
                movie_data.append(movie_info)
        except Exception as e:
            print(f"Error processing movie {movie_id}: {e}")

    # Save the movie data to a JSON file
    json_filename = os.path.join(result_folder, f"movie_data_{year}.json")
    write_json(movie_data, json_filename)

    # Load the JSON data into a DataFrame (assuming the structure is correct)
    df = pd.DataFrame(movie_data)

    # Save the DataFrame to a compressed CSV file
    csv_filename = os.path.join(result_folder, f"movie_data_{year}.csv.gz")
    df.to_csv(csv_filename, sep='\t', compression='gzip', index=False)

    # Append the movie data to the list
    all_movie_data.extend(movie_data)

# Handle the case where the movie information is not found
def get_movie_data(movie_id):
    try:
        movie = tmdb.Movies(movie_id)
        info = movie.info()
        return info
    except Exception as e:
        print(f"Error getting movie data for movie {movie_id}: {e}")
        return None


Error getting movie data for movie tt0115937: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0115937?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0116628: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0116628?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0116847: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0116847?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0118710: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0118710?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0120673: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0120673?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0120729: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0120729?api_key=10eef636b9c9ea3efa6

Error getting movie data for movie tt0193469: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0193469?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0194942: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0194942?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0196573: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0196573?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0198460: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0198460?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0198970: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0198970?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0201275: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0201275?api_key=10eef636b9c9ea3efa6

Error getting movie data for movie tt0211526: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0211526?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0211953: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0211953?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0212774: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0212774?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0212873: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0212873?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0213464: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0213464?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0213474: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0213474?api_key=10eef636b9c9ea3efa6

Error getting movie data for movie tt0228309: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0228309?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0228389: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0228389?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0228414: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0228414?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0228417: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0228417?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0228504: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0228504?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0228532: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0228532?api_key=10eef636b9c9ea3efa6

Error getting movie data for movie tt0241249: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0241249?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0241250: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0241250?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0241297: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0241297?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0241729: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0241729?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0241949: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0241949?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0242339: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0242339?api_key=10eef636b9c9ea3efa6

...

Error getting movie data for movie tt0365707: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0365707?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0365711: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0365711?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0365858: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0365858?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0368360: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0368360?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0368918: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0368918?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0368921: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0368921?api_key=10eef636b9c9ea3efa6

Error getting movie data for movie tt0449530: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0449530?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0452828: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0452828?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0454601: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0454601?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0457592: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0457592?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0462022: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0462022?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt0468735: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0468735?api_key=10eef636b9c9ea3efa6

Error getting movie data for movie tt15415112: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt15415112?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt1543821: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt1543821?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt1568879: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt1568879?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt15767310: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt15767310?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt1661228: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt1661228?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt1795679: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt1795679?api_key=10eef636b9c9ea3

Error getting movie data for movie tt27753093: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt27753093?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt27991525: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt27991525?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt28445080: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt28445080?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt2866332: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt2866332?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt28803064: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt28803064?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt2890320: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt2890320?api_key=10eef636b9c

Error getting movie data for movie tt8290142: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt8290142?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt8369902: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt8369902?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt8546902: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt8546902?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt8629376: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt8629376?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt8751792: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt8751792?api_key=10eef636b9c9ea3efa6bb19d8e33d397
Error getting movie data for movie tt8846956: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt8846956?api_key=10eef636b9c9ea3efa6